Install TensorFlow Version 1

In [ ]:
!pip install 'tensorflow==1.6.0'
!pip install 'keras==2.2.4'

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import Required Packages

In [ ]:
import os
from os import listdir
import sys
import json
import random
import skimage.io
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/Project/Mask_RCNN-master/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.utils import Dataset
import mrcnn.model as modellib
from mrcnn import utils,visualize
from mrcnn.model import log
from mrcnn.model import load_image_gt
from mrcnn.config import Config
from mrcnn.model import mold_image
from mrcnn.utils import compute_ap
from mrcnn.utils import extract_bboxes

#ROOT_DIR2 = os.path.abspath("/content/drive/MyDrive/Project/Mask_RCNN-master/Mask_RCNN-master/samples")
#sys.path.append(ROOT_DIR2)
#from coco import coco

Save some Variables that will be used later

In [ ]:
Num_Classes = 5
Weight_File = '/content/drive/MyDrive/Project/Mask_RCNN-master/Mask_RCNN-master/logs/final_model20201209T2024/mask_rcnn_final_model_0010.h5'
#Train_Annot_File = "/content/drive/MyDrive/Project/Sets/Testing/test_coco.json"
#Train_Annot_Image_Dir = "/content/drive/MyDrive/Project/Sets/Testing"
Test_Image_Dir = "/content/drive/MyDrive/Project/Sets/Testing"
Model_Name = "test_model"

In [ ]:
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Override Config for Custom Testing

In [ ]:
class TestConfig(Config):
    # define the name of the configuration
    NAME = 'drone_cfg'
    # number of classes (background + others)
    NUM_CLASSES = 1 + 5
    # number of training steps per epoch
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH=150
    
    
    ###################################################################################
    BACKBONE = "resnet50"
    IMAGES_PER_GPU = 1
    
    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.6
    
    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # Percent of positive ROIs used to train classifier/mask heads
    ROI_POSITIVE_RATIO = 0.6
    
#     IMAGE_MIN_DIM = 512
#     IMAGE_MAX_DIM = 1024
    
    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    #TRAIN_ROIS_PER_IMAGE = 512
    
    # Max number of final detections
    DETECTION_MAX_INSTANCES = 100
    
    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.8

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3
    pass
 
# prepare config
config = TestConfig()
config.display()

Create the load_dataset and load_mask Functions

In [ ]:
class Test_Dataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """Load the dataset.
        """
        # Add classes
        self.add_class("dataset", 1, "Head")
        self.add_class("dataset", 2, "Thread")
        self.add_class("dataset", 3, "Nut")
        self.add_class("dataset", 4, "Washer")
        self.add_class("dataset", 5, "Pin")

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join('/content/drive/MyDrive/Project/Sets/', "new_annotation.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
      
        # Add images
        for a in annotations:
            
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes 
            
            polygons = [r['shape_attributes'] for r in a['regions']] 
            category = [c['region_attributes']['class'] for c in a['regions']]
            classes = {"Head":1,"Thread":2,"Nut":3,"Washer":4,"Pin":5}    
            num_ids = [classes[a] for a in category]
            
            

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "dataset",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: array of class IDs of the instance masks.
        """
        # If not a dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "dataset":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        
        return mask.astype(np.bool), np.array(num_ids, dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "dataset":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

Create a Test Dataset

In [ ]:
test_set = Test_Dataset()
test_set.load_dataset(Test_Image_Dir)
test_set.prepare()

Create an Array with the different Class Names

In [ ]:
class_names = ['BG', 'Head', 'Thread', 'Nut', 'Washer', 'Pin']

Create a Model that will inference Object Detection

In [ ]:
model = modellib.MaskRCNN(mode = 'inference', config = TestConfig(), model_dir = MODEL_DIR)

Upload the Weight File that was created by Training.py

In [ ]:
model.load_weights(Weight_File, by_name=True,)
#model.load_weights(model.find_last(), by_name=True)

Display the Image without any Annotations for Comparison

In [ ]:
file_names = next(os.walk(Test_Image_Dir))[2]
image = skimage.io.imread(os.path.join(Test_Image_Dir, 'img_34_5_2.jpg'))

skimage.io.imshow(image)
plt.show()

Visualize Ground Truth

In [ ]:
mask, class_ids = test_set.load_mask(24)
bbox = extract_bboxes(mask)
visualize.display_instances(image, bbox, mask, class_ids, test_set.class_names)

Have the Model Detect Objects and Display the Result

In [ ]:
#file_names = next(os.walk(Test_Image_Dir))[2]
#image = skimage.io.imread(os.path.join(Test_Image_Dir, random.choice(file_names)))
image = skimage.io.imread(os.path.join(Test_Image_Dir, 'img_34_5_2.jpg'))
results = model.detect([image],verbose=0)
r = results[0]
#print(r)
visualize.display_instances(image, r['rois'],r['masks'], r['class_ids'],class_names,r['scores'])

Calculate mAP Score

In [ ]:
image_ids = np.random.choice(test_set.image_ids, len(test_set.image_ids))

APs = []

for id in range(len(image_ids)):
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    load_image_gt(test_set, config, image_ids[id], use_mini_mask=False)

  molded_images = np.expand_dims(mold_image(image,config),0)
  results = model.detect([image],verbose = 0)
  r = results[0]
  AP, precisions, recalls, overlaps =\
    compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'],r['class_ids'],r['scores'],r['masks'])
  APs.append(AP)
  print(AP)

print("mAP: ", np.mean(APs))